Expiriment to get mypy to play ball with the snippet testing strategy used for pyright


In [1]:
import os
from pathlib import Path

notebook_path = os.getcwd()
check_path = Path(notebook_path, "../tests/quality_tests/check_esp32").resolve().absolute()

print("Notebook Path:", notebook_path)
print("check_path:", check_path)

Notebook Path: c:\develop\MyPython\micropython-stubs\scripts
check_path: C:\develop\MyPython\micropython-stubs\tests\quality_tests\check_esp32


In [2]:
# basic install with the latest local stdlib and esp32 stubs

import shutil

if os.path.exists(check_path / "typings"):
    shutil.rmtree(check_path / "typings")

os.chdir(notebook_path)
%pip install --target {check_path}/typings --no-user "../publish/micropython-stdlib-stubs" micropython-esp32-stubs
# %pip install --target {check_path}/typings --no-user micropython-esp32-stubs

Processing c:\develop\mypython\micropython-stubs\publish\micropython-stdlib-stubsNote: you may need to restart the kernel to use updated packages.

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached micropython_esp32_stubs-1.21.0.post4-py3-none-any.whl.metadata (3.2 kB)
Using cached micropython_esp32_stubs-1.21.0.post4-py3-none-any.whl (147 kB)
  Created wheel for micropython-stdlib-stubs: filename=micropython_stdlib_stubs-1.1.1a3-py3-none-any.whl size=121779 sha256=8f415fb4ae0d88647b3a36012816c46560f8a9854119a08f3380a3596691711a
  Stored in directory: c:\users\josverl\appdata\local\pip\cache\wheels\dc\bf\64\badaeca3b642752745d83261430f6134cd4a4216f273387121
Successfully built micropython-stdlib-s

In [6]:
import subprocess
import json
from mypy_gitlab_code_quality import generate_report as gitlab_report

def mypy_patch(check_path):
    if not check_path.exists():
        Exception(f"Path {check_path} not found")

    for f in ("typings/sys.pyi",):
        file = check_path / f

        if file.exists():
            print(f"Removing {f}")
            if file.is_file():
                file.unlink()

            elif file.is_dir():
                shutil.rmtree(file)


def run_mypy(path: Path):
    print(f"Running mypy in {path}")
    mypy_patch(path)
    cmd = ["mypy", "-v", "."]
    cmd = ["mypy", "--no-error-summary", "--no-color", "."]
    try:
        output = subprocess.run(
            cmd,
            # check=True,
            cwd=path,
            capture_output=True,
            text=True,
            shell=False,
        )
        return output.stdout
    except subprocess.CalledProcessError as e:
        print(e)

# convert from gitlab to pyright format

HEADER = """
{
    "version": "",
    "time": "",
    "generalDiagnostics": [],
    "summary": {
        "filesAnalyzed": -1,
        "errorCount": -1,
        "warningCount": -1,
        "informationCount": -1,
        "timeInSec": -1
    }
}
"""
DIAGNOSTIC = """
{
    "file": "",
    "severity": "",
    "message": "",
    "rule": "",
    "range": {
        "start": {
            "line": -1,
            "character": -1
        },
        "end": {
            "line": -1,
            "character": -1
        }
    }
}
"""

def gitlab_to_pyright(report):
    pyright_report = json.loads(HEADER)
    pyright_report["version"] = "1.0"
    pyright_report["generalDiagnostics"] = []
    for issue in report:
        i = json.loads(DIAGNOSTIC)
        i["file"] = issue["location"]["path"]
        i["severity"] = issue["severity"] # TODO: map to pyright severity
        i["message"] = issue["description"]
        i["rule"] = issue["check_name"]
        i["range"]["start"]["line"] = issue["location"]["lines"]["begin"]
        pyright_report["generalDiagnostics"].append(i)

    pyright_report["summary"]["errorCount"] = len(pyright_report["generalDiagnostics"])
    return pyright_report



In [9]:
# run mypy
raw_results = run_mypy(check_path)
results = raw_results.split("\n")

gitlab_to_pyright(gitlab_report(map(str.rstrip, results)))

Running mypy in C:\develop\MyPython\micropython-stubs\tests\quality_tests\check_esp32


{'version': '1.0',
 'time': '',
 'generalDiagnostics': [{'file': 'check_aioespnow.py',
   'severity': <Severity.major: 'major'>,
   'message': 'Cannot instantiate abstract class "AIOESPNow" with abstract attribute "__next__"',
   'rule': 'abstract',
   'range': {'start': {'line': 14, 'character': -1},
    'end': {'line': -1, 'character': -1}}}],
 'summary': {'filesAnalyzed': -1,
  'errorCount': 1,
  'warningCount': -1,
  'informationCount': -1,
  'timeInSec': -1}}